# A closer look into Qiskit Nature's modularity

This tutorial showcases the power and flexibility of the different components that make up Qiskit Nature.
Qiskit Nature is designed to enable both end-to-end and modular workflows, where each individual step
can be customized to your particular problem setting. This modularity also allows users to leverage the
full capabilities of Qiskit.

## Using VQE to solve an Electronic Structure Problem

### 1. Obtaining an initial Hartree-Fock solution

Qiskit Nature can interface with different classical codes which are able to find the HF solutions:

- Gaussian
- Psi4
- PySCF

In the following example, we set up a PySCF driver for the hydrogen molecule at equilibrium bond length
(0.735 angstrom) in the singlet state and with no charge.

Running this driver will yield an `ElectronicStructureProblem`, Qiskit Nature's representation of the
electronic structure problem which we are interested in solving. This problem class includes a representation
of the `ElectronicEnergy` hamiltonian, which can be extracted using the `hamiltonian` property:


In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

hamiltonian = driver.run().hamiltonian
print(hamiltonian)

## The ElectronicEnergy Hamiltonian

This hamiltonian object stores the following information:

In [2]:
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

Polynomial Tensor
 "+-":
[[-1.25633907e+00 -1.37083854e-17]
 [-3.92720783e-17 -4.71896007e-01]]
 "++--":
[[[[6.75710155e-01 8.55877331e-17]
   [1.08243975e-16 1.80931200e-01]]

  [[6.56516662e-17 1.80931200e-01]
   [6.64581730e-01 2.44635896e-16]]]


 [[[4.88822879e-17 6.64581730e-01]
   [1.80931200e-01 1.58172569e-16]]

  [[1.80931200e-01 3.47090768e-16]
   [3.21674662e-17 6.98573723e-01]]]]


In [3]:
nuclear_repulsion_energy = hamiltonian.nuclear_repulsion_energy
print(nuclear_repulsion_energy)

0.7199689944489797


This class is also able to generate the second-quantized operator (fermionic operator) from the 1- and 2-body integrals which the classical code has computed for us.

In [4]:
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.3378550774017583 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211575 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211567 * ( +_0 +_1 -_0 -_1 )
+ 0.3322908651276485 * ( +_0 +_1 -_1 -_0 )
+ 0.3378550774017583 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211575 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211567 * ( +_0 +_3 -_2 -_1 )
+ 0.3322908651276485 * ( +_0 +_3 -_3 -_0 )
+ 0.33229086512764827 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211575 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211568 * ( +_1 +_1 -_0 -_0 )
+ 0.3492868613660089 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764827 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211575 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211568 * ( +_1 +_3 -_2 -_0 )
+ 0.3492868613660089 * ( +_1 +_3 -_3 -_1 )
+ 0.3378550774017583 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211575 * ( +_2 +_0 -_1 -_3 )
+ 0.0904655998

Running the driver is not the only way to build an Electronic Energy hamiltonian, it can also be  defined manually from the raw electronic integrals as numpy arrays, or from instances of the `ElectronicIntegral` class.

In [5]:
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
import numpy as np

other_hamiltonian = ElectronicEnergy.from_raw_integrals(np.zeros((2, 2)), np.zeros((2, 2, 2, 2)))
other_hamiltonian.nuclear_repulsion_energy = 1.23
print(other_hamiltonian.electronic_integrals.alpha)

Polynomial Tensor
 "+-":
[[0. 0.]
 [0. 0.]]
 "++--":
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]


Additional functionality of this hamiltonian class: computing the coulomb term, exchange term and fock operator for a given reduced density matrix (expressed using the ElectronicIntegrals operator)


In [6]:
from qiskit_nature.second_q.operators import ElectronicIntegrals, PolynomialTensor

density = ElectronicIntegrals(
    alpha=PolynomialTensor({"+-": 0.5 * np.eye(2)}),
    beta=PolynomialTensor({"+-": 0.25 * np.eye(2)}),
)
fock_op = hamiltonian.fock(density)
print("f", fock_op.alpha)
coulomb = hamiltonian.coulomb(density)
print("c", coulomb.alpha)
exchange = hamiltonian.exchange(density)
print("e", exchange.alpha)

f Polynomial Tensor
 "+-":
array([[-3.44367865e-01,  7.62682226e-17],
       [ 1.75483828e-16,  4.51506984e-01]])
c Polynomial Tensor
 "+-":
array([[1.34029189e+00, 2.66416544e-16],
       [4.12742434e-16, 1.36315545e+00]])
e Polynomial Tensor
 "+-":
array([[4.28320677e-01, 1.76439936e-16],
       [1.97986528e-16, 4.39752461e-01]])


## Going from Hamiltonian to Fermionic Operator

As introduced above, the Qiskit Nature `Hamiltonian` subclasses have a `.second_q_op()` method that converts them to a corresponding second quantized operator (Fermionic, Bosonic or Spin, depending on the origin hamiltonian type).

In [7]:
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.3378550774017583 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211575 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211567 * ( +_0 +_1 -_0 -_1 )
+ 0.3322908651276485 * ( +_0 +_1 -_1 -_0 )
+ 0.3378550774017583 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211575 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211567 * ( +_0 +_3 -_2 -_1 )
+ 0.3322908651276485 * ( +_0 +_3 -_3 -_0 )
+ 0.33229086512764827 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211575 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211568 * ( +_1 +_1 -_0 -_0 )
+ 0.3492868613660089 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764827 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211575 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211568 * ( +_1 +_3 -_2 -_0 )
+ 0.3492868613660089 * ( +_1 +_3 -_3 -_1 )
+ 0.3378550774017583 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211575 * ( +_2 +_0 -_1 -_3 )
+ 0.0904655998

These operators can always be constructed manually from a descriptive string:


In [8]:
from qiskit_nature.second_q.operators import FermionicOp

op1 = FermionicOp({"+_0 -_0": -1}, num_spin_orbitals=1)
print(op1)

Fermionic Operator
number spin orbitals=1, number terms=1
  -1 * ( +_0 -_0 )


In [9]:
op2 = FermionicOp({"+_0 -_0": 1, "-_0 +_0": 2})
print(op2)

Fermionic Operator
number spin orbitals=1, number terms=2
  1 * ( +_0 -_0 )
+ 2 * ( -_0 +_0 )


In [10]:
h2_labels = {
    "+_0 -_1 +_2 -_3": 0.18093120148374142,
    "+_0 -_1 -_2 +_3": -0.18093120148374134,
    "-_0 +_1 +_2 -_3": -0.18093120148374134,
    "-_0 +_1 -_2 +_3": 0.18093120148374128,
    "+_3 -_3": -0.4718960038869427,
    "+_2 -_2": -1.2563391028292563,
    "+_2 -_2 +_3 -_3": 0.48365053378098793,
    "+_1 -_1": -0.4718960038869427,
    "+_1 -_1 +_3 -_3": 0.6985737398458793,
    "+_1 -_1 +_2 -_2": 0.6645817352647293,
    "+_0 -_0": -1.2563391028292563,
    "+_0 -_0 +_3 -_3": 0.6645817352647293,
    "+_0 -_0 +_2 -_2": 0.6757101625347564,
    "+_0 -_0 +_1 -_1": 0.48365053378098793,
}

h2_op = FermionicOp(h2_labels, num_spin_orbitals=4)
print(h2_op)

Fermionic Operator
number spin orbitals=4, number terms=14
  0.18093120148374142 * ( +_0 -_1 +_2 -_3 )
+ -0.18093120148374134 * ( +_0 -_1 -_2 +_3 )
+ -0.18093120148374134 * ( -_0 +_1 +_2 -_3 )
+ 0.18093120148374128 * ( -_0 +_1 -_2 +_3 )
+ -0.4718960038869427 * ( +_3 -_3 )
+ -1.2563391028292563 * ( +_2 -_2 )
+ 0.48365053378098793 * ( +_2 -_2 +_3 -_3 )
+ -0.4718960038869427 * ( +_1 -_1 )
+ 0.6985737398458793 * ( +_1 -_1 +_3 -_3 )
+ 0.6645817352647293 * ( +_1 -_1 +_2 -_2 )
+ -1.2563391028292563 * ( +_0 -_0 )
+ 0.6645817352647293 * ( +_0 -_0 +_3 -_3 )
+ 0.6757101625347564 * ( +_0 -_0 +_2 -_2 )
+ 0.48365053378098793 * ( +_0 -_0 +_1 -_1 )


Fermionic Operators support algebraic operations such as addition, adjoint, scalar operations...


In [11]:
op3 = op1 + op2
print(op3)

Fermionic Operator
number spin orbitals=1, number terms=2
  0 * ( +_0 -_0 )
+ 2 * ( -_0 +_0 )


In [12]:
op4 = (2 + 1j) * op3

print(op4.adjoint())

Fermionic Operator
number spin orbitals=1, number terms=2
  -0j * ( +_0 -_0 )
+ (4-2j) * ( -_0 +_0 )


## Mapping the FermionicOp into a Qubit Operator

Qiskit Nature offers a series of pre-implemented mappers to convert from "Nature-native operators" to "Qiskit-native operators".
For example, the parity mapper or the Jordan-Wigner mapper.

In [13]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper()
qubit_p_op = mapper.map(fermionic_op)
print(qubit_p_op)

-0.8105479805373275 * IIII
+ 0.1721839326191553 * IIIZ
- 0.22575349222402408 * IIZZ
+ 0.17218393261915538 * IZZI
- 0.22575349222402405 * ZZII
+ 0.12091263261776633 * IIZI
+ 0.16892753870087915 * IZZZ
+ 0.045232799946057854 * ZXIX
- 0.045232799946057854 * IXZX
- 0.045232799946057854 * ZXZX
+ 0.045232799946057854 * IXIX
+ 0.1661454325638242 * ZZIZ
+ 0.1661454325638242 * IZIZ
+ 0.17464343068300445 * ZZZZ
+ 0.12091263261776633 * ZIZI


/opt/anaconda3/envs/nature/lib/python3.9/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


In [14]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

-0.8105479805373275 * IIII
+ 0.1721839326191553 * IIIZ
- 0.22575349222402408 * IIZI
+ 0.17218393261915538 * IZII
- 0.22575349222402405 * ZIII
+ 0.12091263261776633 * IIZZ
+ 0.16892753870087915 * IZIZ
+ 0.045232799946057854 * YYYY
+ 0.045232799946057854 * XXYY
+ 0.045232799946057854 * YYXX
+ 0.045232799946057854 * XXXX
+ 0.1661454325638242 * ZIIZ
+ 0.1661454325638242 * IZZI
+ 0.17464343068300445 * ZIZI
+ 0.12091263261776633 * ZZII


## Plugging this operator into any generic VQE workflow

The qubit operators outputted by the mappers are standard Qiskit operators of type `qiskit.quantum_info.SparsePauliOp`,
which allow for a seamless integration into any standard algorithm written using Qiskit, for example, an implementation
of VQE using the Qiskit primitives.

In [15]:
# Define ansatz
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(qubit_jw_op.num_qubits)
num_params = ansatz.num_parameters

In [16]:
# Define cost function
def cost_func(params, ansatz, hamiltonian, estimator):
    energy = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return energy

In [17]:
# Define intial point and primitive instance
from qiskit.primitives import Estimator

x0 = 2 * np.pi * np.random.random(num_params)
estimator = Estimator(options={"shots": int(1e4)})

In [18]:
# Minimize cost function using a SciPy optimizer
from scipy.optimize import minimize

res = minimize(cost_func, x0, args=(ansatz, qubit_jw_op, estimator), method="cobyla")
print("Ground state energy: ", res.fun)

Ground state energy:  -1.8040668692818573


# Full Workflow Summarized in a Single Cell


In [19]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator

from scipy.optimize import minimize

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# get molecular hamiltonian from PySCF driver
electr_energy_hamiltonian = driver.run().hamiltonian

# convert molecular hamiltonian to fermionic operator
fermionic_op = electr_energy_hamiltonian.second_q_op()

# map fermionic operator into qubit operator
# of type qiskit.quantum_info.SparsePauliOp
mapper = JordanWignerMapper()
qubit_op = mapper.map(fermionic_op)

# this operator can now be plugged into any
# qiskit workflow, for example....

# find ground state energy with VQE
# using the Estimator primitive
estimator = Estimator(options={"shots": int(1e4)})

# define ansatz and initial point
ansatz = EfficientSU2(qubit_op.num_qubits)
num_params = ansatz.num_parameters
initial_point = 2 * np.pi * np.random.random(num_params)

# define VQE cost function
def cost_func(params, ansatz, operator, estimator):
    energy = estimator.run(ansatz, operator, parameter_values=params).result().values[0]
    return energy


# run SciPy minimizer to find ground state
result = minimize(cost_func, initial_point, args=(ansatz, qubit_jw_op, estimator), method="cobyla")
print("Ground State Energy:", result.fun)


Ground State Energy: -1.8318022304956496
